In [1]:
import os, sys
import json
from IPython.display import HTML
from IPython.display import clear_output

import requests
import warnings
warnings.filterwarnings('ignore')

import getpass

from pathlib import Path
from hda import Client, Configuration

In [2]:
hdarc = Path(Path.home() / '.hdarc')

# Create it only if it does not already exists
if not hdarc.is_file():
    import getpass
    USERNAME = input('Enter your username: ')
    PASSWORD = getpass.getpass('Enter your password: ')

    with open(Path.home() / '.hdarc', 'w') as f:
        f.write(f'user:{USERNAME}\n')
        f.write(f'password:{PASSWORD}\n')

In [3]:
hda_client = Client()

In [4]:
VEGETATION_INDICES='EO:EEA:DAT:CLMS_HRVPP_VPP-LAEA'
SEASONAL_TRAJECTORIES='EO:EEA:DAT:CLMS_HRVPP_ST-LAEA'
CLMS_data = SEASONAL_TRAJECTORIES

In [5]:
hda_client.dataset(CLMS_data)

{'dataset_id': 'EO:EEA:DAT:CLMS_HRVPP_ST-LAEA',
 'terms': ['Copernicus_Land_Monitoring_Service_Data_Policy'],
 'roles': [],
 'multisearch': False,
 'status': 'enabled',
 'metadata': {'_index': 'csw',
  '_id': 'EO:EEA:DAT:CLMS_HRVPP_ST-LAEA',
  '_score': 1.0,
  '_source': {'abstract': 'The Seasonal Trajectories product is a filtered time series of Plant Phenology Index (PPI), with regular time step, part of the Copernicus Land Monitoring Service (CLMS) HR-VPP product suite. \n\nPlant Phenology Index (PPI) is a physically based vegetation index that is optimized for monitoring vegetation phenology and has a linear relationship with green leaf area index. \n\nThe Seasonal Trajectories (ST) products are derived from a function fitting of the time series of the PPI vegetation index and hence provide a filtered PPI time series, with regular 10-day time step, together with related quality information (QFLAG).\n\nThe PPI Seasonal Trajectories are derived from the PPI vegetation index and hence

In [6]:
s=hda_client.metadata(dataset_id=CLMS_data)
print(json.dumps(s,indent=2))

{
  "type": "object",
  "title": "Queryable",
  "properties": {
    "dataset_id": {
      "title": "dataset_id",
      "type": "string",
      "oneOf": [
        {
          "const": "EO:EEA:DAT:CLMS_HRVPP_ST-LAEA",
          "title": "EO:EEA:DAT:CLMS_HRVPP_ST-LAEA",
          "group": null
        }
      ]
    },
    "itemsPerPage": {
      "title": "Items PerPage",
      "type": "string",
      "pattern": "^[0-9]*$"
    },
    "startIndex": {
      "title": "Start Index",
      "type": "string",
      "pattern": "[1-9][0-9]*"
    },
    "httpAccept": {
      "title": "Http Accept",
      "type": "string",
      "oneOf": [
        {
          "const": "application%2Fatom%2Bxml",
          "title": "Atom",
          "group": null
        },
        {
          "const": "application%2Fgeo%2Bjson",
          "title": "GeoJson",
          "group": null
        }
      ]
    },
    "recordSchema": {
      "title": "Record Schema",
      "type": "string",
      "oneOf": [
        {
       

"MINV", "MAXD", "LENGTH", "SOSD", "QFLAG", "EOSV", "TPROD", "MAXV", "AMPL", "SOSV", "LSLOPE", "EOSD", "RSLOPE", "SPROD"


MINV, MAXD, EOSV, TPROD

In [7]:
# Define the variable
product_type = "QFLAG"

# Define the JSON structure as a Python dictionary
st_query = {
    "datasetId": CLMS_data,
    "boundingBoxValues": [
        {
            "name": "bbox",
            "bbox": [
                9.42,
                46.38,
                17.18,
                49.04
            ]
        }
    ],
    "dateRangeSelectValues": [
        {
            "name": "temporal_interval",
            "start": "2019-01-01T00:00:00.000Z",
            "end": "2019-12-31T00:00:00.000Z"
        }
    ],
    "stringChoiceValues": [
        {
            "name": "productType",
            "value": product_type
        }
    ]
}

# Print the JSON as a pretty string
print(json.dumps(st_query, indent=2))

{
  "datasetId": "EO:EEA:DAT:CLMS_HRVPP_ST-LAEA",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        9.42,
        46.38,
        17.18,
        49.04
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "temporal_interval",
      "start": "2019-01-01T00:00:00.000Z",
      "end": "2019-12-31T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "productType",
      "value": "QFLAG"
    }
  ]
}


In [8]:
matches = hda_client.search(st_query)
print(matches)

SearchResults[items=1080,volume=9.5GB]


In [9]:
# path = f"../../eodc/private/openeo_platform/zarr_nacho/VPP_{product_type}"
path = f"../../eodc/private/tempearth/CLMS/ST_{product_type}"
# path = "home/koenifra/VPP_TPROD"
path


'../../eodc/private/tempearth/CLMS/ST_QFLAG'

In [10]:
matches = matches[131:]
matches

SearchResults[items=949,volume=7.5GB]

In [11]:
matches.download(path)